# **For Downloading a dataset for testing**

In [ ]:

!git clone https://github.com/ailab-uniud/akec.git



# **Testing Connection with Cloud**

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

# **Downloading the Needed Libraries for the code to run properly**


1.   Gensim for Word2Vec model loading download from github https://github.com/bakrianoo/aravec , Twitter 300 CBOW.
Found on the drive link :
https://drive.google.com/open?id=1ubnmvIOqDLgoK7zZu-_UDvNwqMmvLcWO
2.   Keras for the deep learning models and text preprocessing
3.   Sklearn for the evaluation of the model metrics
4.   UnicodeCsv for reading and editing the arabic language DataSet







In [ ]:
!pip install gensim
!pip install keras
!pip install sklearn
!pip install unicodecsv


# The Following Two Cells responsible for the connection with your own drive where the data is loaded for people who use Google Colab where upload and download is much faster

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


In [ ]:
!mkdir -p drive
!google-drive-ocamlfuse drive


# **The imported Libraries and packages to be used in the whole notebook**

In [ ]:

from gensim.models import Word2Vec
import random
import unicodecsv as csv
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from keras.models import Sequential,Model,model_from_json
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Flatten, TimeDistributed,Conv1D,MaxPooling1D,Input
from keras.layers import Embedding
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.utils import to_categorical

# Total is an array where the whole dataset is loaded 
Opening the file with read byte type for arabic language and encoding utf-8, errors=ignore for strange characters to continue readng the file and donot break.
The process in appending is made 0:2 only taking first two columns as some documents exceed the limit of CSV file format and take more than two cells , all with category economics as humanly observed and revised. This problem will be dealt with soon

In [ ]:
Total=[]

with open(u'drive/LSTM-DS.csv', "rb") as csvfile:
    reader = csv.reader(csvfile, encoding="utf-8",errors='ignore')
    

    for row in reader:
       
       Total.append(row[0:2]) 

random.shuffle(Total)
print(len(Total))

# Making sure data is read correctly with no errors

In [ ]:
print(Total[0])

# Excluding sequences that has more than 80 words and less than 5.

In [ ]:
for line in Total:
  if len(line[0].split(" "))>80:
    Total.remove(line)
  if len(line[0].split(" "))<5:
    Total.remove(line)
  


# **Removing Lines with no labels then appending the sequences to dataset array and labels into labels array**

In [ ]:
dataset=[]
labels=[]
for line in Total:
  if len(line[1].split(" "))>1:
    Total.remove(line)
  else:
    dataset.append(line[0])
    labels.append(line[1])

# **Spliting the sequences into words for training , as each word will correspond to a vector, then printing the total size after preprocessing**

In [ ]:
X_total=[i.split(" ") for i in dataset]
print(len(Total))
print(len(dataset))

# **Defining the maximum number of different words available and the tokenizer package in keras which allows us to map words into their position in the whole dataset which is easier to retrieve and faster to process **

In [ ]:
MAX_NB_WORDS=495145
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(X_total)
sequences = tokenizer.texts_to_sequences(X_total)

# **Defining the number of labels we have , 5 categories and mapping the labels to numbers**

In [ ]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
label=[]
label_index={'diverse':0,'news':1,'sports':2,'economics':3,'culture':4}

for word in labels:
 
  if label_index.get(word):
    label_id=label_index[word]
    label.append(label_id)
  else:
    label_id=3
    label.append(label_id)


# **Padding the sequences to all have the same sequence of 80 words maximum per sequence. Then printing the shape of our dataset**

In [ ]:


MAX_SEQUENCE_LENGTH=80
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH,padding='post')
label = to_categorical(np.asarray(label))

print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', label.shape)
print(label[:10])

# Again Making sure everything is working :D

In [ ]:
print(data[0])

# **Spliting the data into training data and validation data with 80% , 20% portion respectively**

In [ ]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
label = label[indices]
nb_validation_samples = int(0.2 * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = label[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = label[-nb_validation_samples:]
print(y_train.shape)

# **Loading Word2Vec model and creating the embedding matrix which is mapped index to word and word to index**

In [ ]:
modelwv=Word2Vec.load(u"drive/tweet_cbow_300/tweet_cbow_300/tweets_cbow_300")

embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    if word in modelwv.wv.vocab:
      embedding_vector = modelwv[word]
    else:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector



# Lastly checking the dimensions of embedding matrix

In [ ]:
print(embedding_matrix.shape)
EMBEDDING_DIM=300

# **Roc Auc Evaluation function but is not very useful in our case**

In [ ]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))



# **Finally our Bidirectional LSTM with conv1D model , specs described in the document passed. Learning process will take a while so please be patient :D**

In [ ]:

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='float32')
embedded_sequences = embedding_layer(sequence_input)
x = SpatialDropout1D(0.2)(embedded_sequences)
x = Bidirectional(LSTM(80, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
x = Conv1D(40, kernel_size = 5, padding = "valid", kernel_initializer = "glorot_uniform")(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool, max_pool]) 
# x = Dense(128, activation='relu')(x)
# x = Dropout(0.1)(x)
preds = Dense(len(label_index), activation='softmax')(x)
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=1e-3),metrics=['accuracy'])
#filepath="drive/weights_base.best.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
#early = EarlyStopping(monitor="val_acc", mode="max", patience=5)
#ra_val = RocAucEvaluation(validation_data=(x_val, y_val), interval = 7)
#callbacks_list = [ra_val,checkpoint, early]
model.fit(x_train, y_train,validation_data=(x_val, y_val), batch_size=100, epochs=4,verbose=1)


# **Another Model : Conv1D model with MaxPooling of 5x5 and relu activation. And saving the model weights and json file to the drive**

In [ ]:


embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='float32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(80, 1, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(80, 1, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(80, 1, activation='relu')(x)
x = MaxPooling1D(2)(x)  # global max pooling
x = Flatten()(x)
x = Dense(80, activation='relu')(x)
preds = Dense(len(label_index)+1, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.fit(x_train, y_train,validation_data=(x_val, y_val),epochs=6,batch_size=50)
scores = model.evaluate(x_train,y_train, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
model_json = model.to_json()
with open("drive/model_conv1d.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("drive/model_conv1dweights.h5")
print("Saved model to drive")

# **Final Model of Bidirectional LSTM**

In [ ]:

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='float32')
embedded_sequences = embedding_layer(sequence_input)
x = Bidirectional(LSTM(80, return_sequences=True, dropout=0.25, recurrent_dropout=0.1))(embedded_sequences)
x = GlobalMaxPool1D()(x)
x = Dense(100, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(len(label_index)+1, activation="sigmoid")(x)
model = Model(inputs=sequence_input, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train,validation_data=(x_val, y_val), batch_size=50, epochs=4,verbose=1) # validation_split=0.1);

# **Evaluation the model performance and printing out the accuracy percentage (the model cell run will have its accuracy printed by using this cell whether its BiLSTM with Conv1D or any other)**

In [ ]:
scores = model.evaluate(x_train,y_train, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
model_json = model.to_json()
with open("drive/final_LSTMCnn.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("drive/final_LSTMCnn_Weights.h5")
print("Saved model to drive")

# **Happy testing, Insert the desired Sentence to be tested in totest**

In [ ]:
MAX_SEQUENCE_LENGTH=80
totest=[""]
totest=[i.split(" ") for i in totest]
print(totest)
Xnew=np.array(tokenizer.texts_to_sequences(totest))
Xnew = pad_sequences(Xnew, maxlen=MAX_SEQUENCE_LENGTH,padding='post')
ynew = model.predict(Xnew)

result=np.argmax(ynew,axis=1)
for k,v in label_index.items():
  if v==result:
    print(k)
